In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)
from bigbio.dataloader import BigBioConfigHelpers
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers_interpret.evaluation.input_pre_processor import InputPreProcessor, get_labels_from_dataset

C:\Users\richa\Documents\GitHub\transformers-interpret


In [3]:
def map_to_string(x):
    return id2label[x]

In [5]:
conhelps = BigBioConfigHelpers()
dataset_name = 'bc5cdr_bigbio_kb'  # 2 classes, short to medium sentence length, Disease
# dataset_name = 'euadr_bigbio_kb'  # 5 classes, short to medium sentence length, Diseases & Disorders
# dataset_name = 'cadec_bigbio_kb'  # 5 classes, shortest documents, forum posts, Disease
# dataset_name = 'scai_disease_bigbio_kb'  # 2 classes, long documents, DISEASE
# dataset_name = 'ncbi_disease_bigbio_kb'
# dataset_name = 'verspoor_2013_bigbio_kb'
dataset = conhelps.for_config_name(dataset_name).load_dataset()

Reusing dataset bc5cdr (C:\Users\richa\.cache\huggingface\datasets\bc5cdr\bc5cdr_bigbio_kb\1.0.0\7e79e79248a93d8b6a31793a47da14c4e8777f17648a27aaa6c6488ac71c8ab0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# huggingface_model = 'dbmdz/electra-large-discriminator-finetuned-conll03-english'
huggingface_model = 'dslim/bert-base-NER'
# huggingface_model = 'Jean-Baptiste/roberta-large-ner-english'

tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(huggingface_model)
additional_tokenizers = []

In [7]:
label2id, id2label = get_labels_from_dataset(dataset)
print(label2id)
pre_processor = InputPreProcessor(tokenizer, additional_tokenizers, label2id)

{'O': 0, 'B-Chemical': 1, 'I-Chemical': 2, 'B-Disease': 3, 'I-Disease': 4}


In [8]:
tokenized_datasets = dataset.map(lambda a: pre_processor(a))

Parameter 'function'=<function <lambda> at 0x000001665EA24EE0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/500 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

In [9]:
dataset = tokenized_datasets["train"]

In [10]:
print(dataset[0])

{'id': '0', 'document_id': '227508', 'passages': [{'id': '1', 'type': 'title', 'text': ['Naloxone reverses the antihypertensive effect of clonidine.'], 'offsets': [[0, 59]]}, {'id': '2', 'type': 'abstract', 'text': ['In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of

In [13]:
for i, doc in enumerate(dataset):
    if i >= 1:
        break
    for token_i, token in enumerate(zip(tokenizer.batch_decode(doc['input_ids']), doc['labels'])):
        if token[0] == '[PAD]':
            break
        print([token_i, token[0], id2label[token[1]]])

[0, '[CLS]', 'O']
[1, 'Na', 'B-Chemical']
[2, '##lo', 'I-Chemical']
[3, '##xon', 'I-Chemical']
[4, '##e', 'I-Chemical']
[5, 'reverse', 'O']
[6, '##s', 'O']
[7, 'the', 'O']
[8, 'anti', 'O']
[9, '##hy', 'O']
[10, '##pert', 'O']
[11, '##ens', 'O']
[12, '##ive', 'O']
[13, 'effect', 'O']
[14, 'of', 'O']
[15, 'c', 'B-Chemical']
[16, '##lon', 'I-Chemical']
[17, '##id', 'I-Chemical']
[18, '##ine', 'I-Chemical']
[19, '.', 'O']
[20, 'In', 'O']
[21, 'un', 'O']
[22, '##ane', 'O']
[23, '##st', 'O']
[24, '##he', 'O']
[25, '##tized', 'O']
[26, ',', 'O']
[27, 'spontaneous', 'O']
[28, '##ly', 'O']
[29, 'h', 'B-Disease']
[30, '##yper', 'I-Disease']
[31, '##tensive', 'I-Disease']
[32, 'rats', 'O']
[33, 'the', 'O']
[34, 'decrease', 'O']
[35, 'in', 'O']
[36, 'blood', 'O']
[37, 'pressure', 'O']
[38, 'and', 'O']
[39, 'heart', 'O']
[40, 'rate', 'O']
[41, 'produced', 'O']
[42, 'by', 'O']
[43, 'in', 'O']
[44, '##tra', 'O']
[45, '##ven', 'O']
[46, '##ous', 'O']
[47, 'c', 'B-Chemical']
[48, '##lon', 'I-Chemical']